In [1]:
import mne
import pandas as pd
import numpy as np
import glob
import pylab
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import os
#*****************************#
### PARAMETERS ###
#*****************************#
pre_stim_time               = -.2
post_stim_time              = 3.2
event_id = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
subjects = [f"S{i:02}" for i in range(1, 21)]
#Asign path to bids directory
bids_top_dir = '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/'
raw_ICA_path = bids_top_dir + 'derivatives/preprocessed/ICA-Removed'

for S in subjects:
    print('Processing Subject...', S)
    stim_duration = .3
    run_length = 4
    first_run_file = 1
    last_run_file = 4
    bids_dir = bids_top_dir + f'bids_dir/sub-{S}/ses-1/meg/sub-{S}_ses-1_task-OrientationImageryDynamic'
    dsets = []
    for i in range(1, 5):
        pattern = f'{raw_ICA_path}/sub-{S}_ses-1_task-OrientationImageryDynamic_run-{i:02}_step1b-raw.fif'
        matches = glob.glob(pattern)
        
        if matches:
            dsets.append(matches[0])
        else:
            print(f"No files found for pattern: {pattern}")
    all_epochs = []
    all_events = []
    all_onsets = []
    all_offsets = []
    for i in dsets:
        raw = mne.io.read_raw_fif(i)
        events = mne.find_events(raw, stim_channel='UPPT001',min_duration = 0.002)
        if len(events) > 200:
            raise ValueError(f"Too many events found in {i}: {len(events)} (max allowed = 200)")
        events = mne.events_from_annotations(raw)[0]
        
        epochs = mne.Epochs(raw, events, event_id=event_id,tmin=pre_stim_time, tmax=post_stim_time, preload=True, baseline=(None,0), picks = 'mag')
        print("Number of events:", len(events))
        print("Number of epochs:", len(epochs))
        print(np.nonzero(list(map(len, epochs.drop_log)))[0])
        [n for n, dl in enumerate(epochs.drop_log) if len(dl)]
        all_epochs.append(epochs)
        all_events.append(events)

    dev_head_t_ref = all_epochs[0].info['dev_head_t']

    for i in range(0, 4):
        all_epochs[i].info['dev_head_t'] = dev_head_t_ref

    epochs_stacked = mne.concatenate_epochs(all_epochs)
    pattern = f'{bids_top_dir}bids_dir/sub-{S}/ses-1/meg/*events.tsv'
    matches = glob.glob(pattern)

    print(len(matches))
    matches.sort()
    out = []

    for event_files in matches[:-4]:
        out.append(pd.read_csv(event_files,sep='\t'))
    all_metadata = pd.concat(out).reset_index(drop=True)
    epochs_stacked.metadata = all_metadata
    runs = [all_epochs[0], all_epochs[1], all_epochs[2], all_epochs[3]] 

    # Get number of trials in each run
    run_lengths = [len(r) for r in runs]

    # Create run number array that matches the actual number of trials per run
    run_nr = np.concatenate([[i] * l for i, l in enumerate(run_lengths)])

    # Assign run numbers to metadata
    epochs_stacked.metadata['run_nr'] = run_nr
    block_type = [k.split('/')[1] for k in epochs_stacked.metadata.trial_type]
    condition = [k.split('/')[2] for k in epochs_stacked.metadata.trial_type]

    # Assign run numbers to metadata
    epochs_stacked.metadata['run_nr'] = run_nr
    epochs_stacked.metadata['block_type'] = block_type
    epochs_stacked.metadata['condition'] = condition
    file_name = f'sub-{S}_Dynamic_preprocessed-epo.fif'
    directory = f'{bids_top_dir}/derivatives/preprocessed/'
    os.makedirs(directory, exist_ok=True)
    # Save
    epochs_stacked.save(f'{directory}{file_name}', overwrite=True)



Processing Subject... S01
Opening raw data file /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/ICA-Removed/sub-S01_ses-1_task-OrientationImageryDynamic_run-01_step1b-raw.fif...
    Read 5 compensation matrices
    Range : 0 ... 589212 =      0.000 ...   491.010 secs
Ready.
Current compensation grade : 3
126 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
Used Annotations descriptions: ['S01/Dynamic/0022_Left', 'S01/Dynamic/0022_Right', 'S01/Dynamic/0067_Left', 'S01/Dynamic/0067_Right', 'S01/Dynamic/0112_Left', 'S01/Dynamic/0112_Right', 'S01/Dynamic/0157_Left', 'S01/Dynamic/0157_Right', 'S01/Dynamic/0202_Left', 'S01/Dynamic/0202_Right', 'S01/Dynamic/0247_Left', 'S01/Dynamic/0247_Right', 'S01/Dynamic/0292_Left', 'S01/Dynamic/0292_Right', 'S01/Dynamic/0337_Left', 'S01/Dynamic/0337_Right', 'S01/Dynamic/catch']
Not setting metadata
126 matching events found
Removing 5 compensators from info because not all

/var/folders/9m/6b08zp213znc3k12g6gpjj9rqx2ly9/T/ipykernel_66090/2286934038.py:61: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs_stacked = mne.concatenate_epochs(all_epochs)


Not setting metadata
504 matching events found
Applying baseline correction (mode: mean)
8
Adding metadata with 5 columns
Overwriting existing file.
Splitting into 2 parts
Processing Subject... S02
Opening raw data file /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/ICA-Removed/sub-S02_ses-1_task-OrientationImageryDynamic_run-01_step1b-raw.fif...
    Read 5 compensation matrices
    Range : 0 ... 559617 =      0.000 ...   466.348 secs
Ready.
Current compensation grade : 3
122 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
Used Annotations descriptions: ['S01/Dynamic/0022_Left', 'S01/Dynamic/0022_Right', 'S01/Dynamic/0067_Left', 'S01/Dynamic/0067_Right', 'S01/Dynamic/0112_Left', 'S01/Dynamic/0112_Right', 'S01/Dynamic/0157_Left', 'S01/Dynamic/0157_Right', 'S01/Dynamic/0202_Left', 'S01/Dynamic/0202_Right', 'S01/Dynamic/0247_Left', 'S01/Dynamic/0247_Right', 'S01/Dynamic/0292_Left', 'S01/Dynamic/0292_Righ

/var/folders/9m/6b08zp213znc3k12g6gpjj9rqx2ly9/T/ipykernel_66090/2286934038.py:61: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs_stacked = mne.concatenate_epochs(all_epochs)


Not setting metadata
488 matching events found
Applying baseline correction (mode: mean)
8
Adding metadata with 5 columns
Processing Subject... S03
Opening raw data file /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/ICA-Removed/sub-S03_ses-1_task-OrientationImageryDynamic_run-01_step1b-raw.fif...
    Read 5 compensation matrices
    Range : 0 ... 573886 =      0.000 ...   478.238 secs
Ready.
Current compensation grade : 3
122 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
Used Annotations descriptions: ['S01/Dynamic/0022_Left', 'S01/Dynamic/0022_Right', 'S01/Dynamic/0067_Left', 'S01/Dynamic/0067_Right', 'S01/Dynamic/0112_Left', 'S01/Dynamic/0112_Right', 'S01/Dynamic/0157_Left', 'S01/Dynamic/0157_Right', 'S01/Dynamic/0202_Left', 'S01/Dynamic/0202_Right', 'S01/Dynamic/0247_Left', 'S01/Dynamic/0247_Right', 'S01/Dynamic/0292_Left', 'S01/Dynamic/0292_Right', 'S01/Dynamic/0337_Left', 'S01/Dynamic/0337_Rig

...

Not setting metadata
489 matching events found
Applying baseline correction (mode: mean)
8
Adding metadata with 5 columns
Splitting into 2 parts
Processing Subject... S19
Opening raw data file /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/ICA-Removed/sub-S19_ses-1_task-OrientationImageryDynamic_run-01_step1b-raw.fif...
    Read 5 compensation matrices
    Range : 0 ... 559343 =      0.000 ...   466.119 secs
Ready.
Current compensation grade : 3
123 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
Used Annotations descriptions: ['S01/Dynamic/0022_Left', 'S01/Dynamic/0022_Right', 'S01/Dynamic/0067_Left', 'S01/Dynamic/0067_Right', 'S01/Dynamic/0112_Left', 'S01/Dynamic/0112_Right', 'S01/Dynamic/0157_Left', 'S01/Dynamic/0157_Right', 'S01/Dynamic/0202_Left', 'S01/Dynamic/0202_Right', 'S01/Dynamic/0247_Left', 'S01/Dynamic/0247_Right', 'S01/Dynamic/0292_Left', 'S01/Dynamic/0292_Right', 'S01/Dynamic/0337_Left'

/var/folders/9m/6b08zp213znc3k12g6gpjj9rqx2ly9/T/ipykernel_66090/2286934038.py:61: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs_stacked = mne.concatenate_epochs(all_epochs)


Not setting metadata
491 matching events found
Applying baseline correction (mode: mean)
8
Adding metadata with 5 columns
Splitting into 2 parts
Processing Subject... S20
Opening raw data file /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/derivatives/preprocessed/ICA-Removed/sub-S20_ses-1_task-OrientationImageryDynamic_run-01_step1b-raw.fif...
    Read 5 compensation matrices
    Range : 0 ... 547443 =      0.000 ...   456.202 secs
Ready.
Current compensation grade : 3
121 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
Used Annotations descriptions: ['S01/Dynamic/0022_Left', 'S01/Dynamic/0022_Right', 'S01/Dynamic/0067_Left', 'S01/Dynamic/0067_Right', 'S01/Dynamic/0112_Left', 'S01/Dynamic/0112_Right', 'S01/Dynamic/0157_Left', 'S01/Dynamic/0157_Right', 'S01/Dynamic/0202_Left', 'S01/Dynamic/0202_Right', 'S01/Dynamic/0247_Left', 'S01/Dynamic/0247_Right', 'S01/Dynamic/0292_Left', 'S01/Dynamic/0292_Right', 'S01/Dynamic/0337_Left'

/var/folders/9m/6b08zp213znc3k12g6gpjj9rqx2ly9/T/ipykernel_66090/2286934038.py:61: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs_stacked = mne.concatenate_epochs(all_epochs)


Not setting metadata
487 matching events found
Applying baseline correction (mode: mean)
8
Adding metadata with 5 columns
